<a href="https://colab.research.google.com/github/salarbalou/steadystate_analysis/blob/main/Itr_vs_CQD_Comcemtration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import requests as req
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import re
import sympy as sy
from sympy import symbols, diff
from sympy.solvers import solve
from scipy.misc import derivative
import string as sr
pio.templates.default ='plotly_dark'
from plotly.subplots import make_subplots
import plotly.express as px

In [2]:
def findmb(x, y): # a simple function to find the slope and intecept of a line
  m,b = np.polyfit(x, y, deg= 1)
  return m,b

In [3]:

CQD_C= ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10'] # a list of CQD Concentartions

all_eta = [] # aan empty list of eta values (photothermal conversion efficiency) that can bes used to store the values of LPS generated in the for loop
all_Itr = []
all_I0 = []
for C in CQD_C: # since we have multiple dutycylce values we make a for loop
 baseurl = 'https://raw.githubusercontent.com/salarbalou/steadystate_analysis/main/CQD' # base url of the text file 
 char = 'X-t-T-dc-Itr.txt' # format of the file
 text = req.get(baseurl+C+char).text # we download and store the text of each file.txt
 data_path = re.compile(r'^([0-9.]+)[\s,]+([0-9.]+)+[\s,]+([-0-9.]+)+[\s,]+([-0-9.]+)+[\s,]+([-0-9.]+)', re.MULTILINE) # we grab the desired text within the text file (all data except for space and comma)
 data_list = data_path.findall(text) # this is a list of quadruple strings: each quadruple in this list have abcd string in it. 
 data = []
 for pentapole in data_list:
      eachpentapole = [] # this list here stores the float version of abcd in each quadruple: eachquadruple is a list of four float values
      for abcde in pentapole:
           eachpentapole.append(float(abcde))
      data.append((eachpentapole[0], eachpentapole[1], eachpentapole[2], eachpentapole[3], eachpentapole[4])) # we append all eachquadruples in data
 final_data = np.array(data) # here we make an array of the list: final_data is a matrix with 183 rows and 4 columns (t, T, I, dc)
 time = final_data[:, 0]
 temperature = final_data[:, 1]
 dutycycle = final_data[:, 2]

 Itr = np.max(final_data[:, 3])/1000 #W
 all_Itr.append(Itr)
 I0 = np.max(final_data[:, 4])/1000 #W
 all_I0.append(I0)



 #I0 = #incident laser power J/s /////Can be increased by increasing laser power
 #Itr = #transmitted laser power J/s ////can be decreased by increasing the concentration AND by overlapping adsorption wl on nanoparticle and lser wl 
 #eta = # photothermal conversion eficiency #coefficient # no units //// is a fucntion of nnaopartcle type
 #h = #heat transfor coefficient between solution and environment+ J/m^2*k*s
 #S = #surface area seperating solution from environemnt +m^2
 #Tw = #solution temp + K
 #T0 = #environment temp + K
 #m = #mass of solition + gr
 #Cp = #specific heat capacity of solution + J/g.K
 #v= volume of the CQD used in microliter + m^3
 #R = heating rate +K/s

 T0 = np.min(temperature)+273.15 #K
 Tw = np.max(temperature)+273.15 #K
 beam_height = 7*1e-3 #radius of the volume filled with CQDs for 10ul CQD #m
 beam_width  = 5*1e-3 # height of the volume filled with CQDs for 10ul CQD #m
 cuvette_lenght = 10*1e-3 #m
 rho = 1000 #density of water kg/m^3
 S= 2*(beam_height+beam_width)*cuvette_lenght #surface area of a part of solution thorugh which the laser beam is passing m^2
 V = beam_height*beam_width*cuvette_lenght #volume of a part of solution thorugh which the laser beam is passing #m^3
 m = (rho*V)*1000 #gr 
 
 h = 600*1e-3 # W/m^2K at 20C #heat transfor coeffcient
 Cp = 4.184 #J/g.K for water
 rate = (Tw-T0)/(np.max(time)-np.min(time))

 eta = (rate*(Cp*m)+np.sum((h*S*(Tw - T0))))/(I0-Itr)*100 # Equation to find the photothermal conversion efficiency #unit less # coefficient in % (can range from 0% to 100%)
 all_eta.append(eta)
 final_eta = np.sum(all_eta)/np.array(all_eta).size

In [ ]:
 fig = make_subplots()
 fig.add_trace(go.Bar(x = np.arange(1,11,1), y = (np.array(all_I0)-np.array(all_Itr))/np.max(time)*1000, marker=dict(color=np.arange(1,11,1))))
 fig2 = make_subplots()
 fig2.add_trace(go.Bar(x = np.arange(1,11,1), y = np.array(all_eta), marker=dict(color='red')))
 fig.update_xaxes(title = 'CQD Concentration (X)', nticks = 20)
 fig.update_yaxes(title = 'Energy Absorbed (mJ)', nticks = 20)
 fig2.update_xaxes(title = 'CQD Concentration (X)', nticks = 20)
 fig2.update_yaxes(title = 'Photothermal Conversion Efficiency (%)', nticks = 20)
 fig.show()
 fig2.show()


## Photothermal Theory:
Plasma is hot matter, in which electron are ripped apart from the 

*   When energy is given to plasma, it starts to oscilliate 
*   this phenomena is called plasmon oscilliation which froms a quasi particle called plasmon


*   in other words, plasmon is the quantum of plasma oscilliaiton 
*   similliar to photons, the quantum particles of optical oscilliaiotn


*   plasmonic resonance is mostly seen in electrons due to the abundacne of free free electron carriers (metals are conductive!)
*   However, plasmonic resonacne can also be seen in semiconductor of nanocrystals (ex,CQDs) that have hih concnetration of free electon carriers. 


*   quantumm dots are artificial atoms, small semiconductor particles
*   when photon couples with electron, electron excites from its valence band to condcution band, which makes a hole in the former.
since the size of the quantum dot is much smaller than the excitation wevelenght, the hole and excited electron can couple and froma quasi particle called exciton. This quasiparticel has high energy compared to the particle which is larger. 








